In [2]:
!git clone -b master https://github.com/charles9n/bert-sklearn

Cloning into 'bert-sklearn'...


In [3]:
import numpy as np
import pandas as pd
import os
import sys
import csv
from sklearn import metrics
from sklearn.metrics import classification_report

sys.path.append("../") 
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
import random 
random.seed(0)
import os
import glob
import pickle
import xgboost, textblob, string
import numpy as np
import pandas as pd
from pandas import read_excel
import itertools 
import matplotlib
matplotlib.use('TkAgg')   
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)

from sklearn.utils import shuffle
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.models import Sequential

Using TensorFlow backend.
09/27/2019 11:43:39 - ERROR - tornado.application -   Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x000001C9E6DA4B88>)
Traceback (most recent call last):
  File "C:\Users\AP84590\AppData\Local\Continuum\anaconda3\lib\site-packages\tornado\ioloop.py", line 743, in _run_callback
    ret = callback()
  File "C:\Users\AP84590\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 314, in advance_eventloop
    eventloop(self)
TypeError: 'NoneType' object is not callable


In [8]:
#Data loading
#Negative cases from disease history notes
CHUSJ_neg = read_excel('data/neg/DiseaseHistory.xlsx', encoding = 'utf8', keep_default_na=False, header=None)
CHUSJ_neg.columns = ['Notes', 'Label']

#Positive cases from disease history notes
CHUSJ_pos_med = read_excel('data/pos/HDmedication.xlsx', encoding = 'utf8', keep_default_na=False, header=None)
CHUSJ_pos_med.columns = ['Notes', 'Label']
CHUSJ_pos_cv = read_excel('data/pos/HDproblemeCV.xlsx', encoding = 'utf8', keep_default_na=False, header=None)
CHUSJ_pos_cv.columns = ['Notes', 'Label']
CHUSJ_pos_bnp = read_excel('data/pos/HDproBNP.xlsx', encoding = 'utf8', keep_default_na=False, header=None)
CHUSJ_pos_bnp.columns = ['Notes', 'Label']

print("Disease history notes for negative cases:")
print(CHUSJ_neg.head())
print ("")
print("Disease history notes for positive cases:")
print ("HD Medication:")
print(CHUSJ_pos_med.head())
print ("")
print ("HD problem CV:")
print(CHUSJ_pos_cv.head())
print ("")
print ("HD proBNP:")
print(CHUSJ_pos_bnp.head())
print ("")
frames = [CHUSJ_neg, CHUSJ_pos_med, CHUSJ_pos_cv, CHUSJ_pos_bnp]
data = pd.concat(frames)

#Randomly shuffle the data
data = shuffle(data)
print('Data after shuffling')
print(data.head(10))

Disease history notes for negative cases:
                                               Notes  Label
0  Appel cette nuit pour détresse respiratoire pr...      0
1  Statut post-op de trachéostomie et reprise cur...      0
2  Pte admise x 11/06 pour dlr thoracique/dyspnée...      0
3  pte présente RGO + dlr thoracique/sensation de...      0
4  Césarienne d'urgence pour détresse fétale le 1...      0

Disease history notes for positive cases:
HD Medication:
                                               Notes  Label
0  PatientName est hospitalisé depuis le 12/05 à ...      1
1  PO immédiat de Lefort (chx mandibule) avec att...      1
2  Sur le plan anesthesique :  induction sufenta/...      1
3  SOP ce jour, Dr Vobecky, dr ruest anesthesie s...      1
4  Rupture amoureuse récente, mais désir de mouri...      1

HD problem CV:
                                               Notes  Label
0  depuis jeudi 01/03, se sent plus fatigué et es...      1
1  Actuellement hospitalisé en cardiologie. 

In [10]:
sentences_train, sentences_test, y_train, y_test = train_test_split(data.Notes, data.Label, test_size=0.4, random_state=0)

In [9]:
from bert_sklearn import SUPPORTED_MODELS
SUPPORTED_MODELS

('bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese')

In [12]:
from sklearn.metrics import matthews_corrcoef

X_train = sentences_train
X_test = sentences_test

# define model
model = BertClassifier(bert_model='bert-base-multilingual-cased')
model.epochs = 3
model.validation_fraction = 0.3
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 2

print('\n',model,'\n')

# fit model
model.fit(X_train, y_train)

# test model on dev

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

# Mathews correlation coefficient
print("\nMathews Correlation: %0.2f"%(matthews_corrcoef(y_test, y_pred) * 100))

Building sklearn classifier...

 BertClassifier(bert_model='bert-base-multilingual-cased', epochs=3,
               eval_batch_size=8, fp16=False, gradient_accumulation_steps=2,
               label_list=None, learning_rate=2e-05, local_rank=-1,
               logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
               num_mlp_hiddens=500, num_mlp_layers=0, random_state=42,
               restore_file=None, train_batch_size=32, use_cuda=True,
               validation_fraction=0.3, warmup_proportion=0.1) 



100%|██████████████████████████████████████████████████████████████████████| 995526/995526 [00:00<00:00, 3499545.48B/s]


Loading bert-base-multilingual-cased model...


100%|████████████████████████████████████████████████████████████████| 662804195/662804195 [01:06<00:00, 9965462.09B/s]


Defaulting to linear classifier/regressor
train data size: 2243, validation data size: 960


...

Accuracy: 83.85%


C:\Users\AP84590\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    negative       0.84      1.00      0.91      1791
    positive       0.00      0.00      0.00       345

    accuracy                           0.84      2136
   macro avg       0.42      0.50      0.46      2136
weighted avg       0.70      0.84      0.76      2136



C:\Users\AP84590\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)



Mathews Correlation: 0.00
